The purpose of this notebook is to investigate how taking small, random subsets of large population affects the betti curves.  Then we can compare with the results of the smaller populations.

In [ ]:
import numpy as np
import neuraltda.topology2 as tp2

import glob
import os

In [ ]:
# Set data and parameters
block_path = '/home/brad/krista/B1083/P03S03/'

# Binning Parameters
windt = 10.0             # milliseconds
dtovr = 0.5*windt        # milliseconds
segment_info = [0, 0]    # use full Trial
cluster_group = ['Good'] # use just good clusters
comment = 'BSPop'        # BootStrap Populations

# Betti Parameters
nperms_subsample = 1                  # 1 permutation for each trial
ncellsperm_subsample = 33             # Subsample 33 neurons from the population
nperms_full = 0                       # For the full population
dims = [0, 1, 2, 3]                   # We want first four betti numbers
thresh = 4.0                          # Activation threshold
twin = np.linspace(0, 6000, 1000)     # milliseconds

In [ ]:
# Bin the data

bfdict = tp2.bin_data(block_path, windt, segment_info,
                      dt_overlap=dtovr, cluster_group=cluster_group, comment=comment)


In [ ]:
# Compute Betti Curves

# Extract paths to binned data
analysis_id = bfdict['analysis_id']
bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]

# Compute betti curves for subsample population
(betti_curves_subsample, t_vals, tvms) = tp2.compute_betti_curves(analysis_id, block_path, bdf,
                                                  thresh, nperms_subsample, ncellsperm_subsample,
                                                  dims, twin, windt, dtovr)

# Compute betti curves for full population
(betti_curves_full, t_vals, tvms) = tp2.compute_betti_curves(analysis_id, block_path, bdf,
                                             thresh, nperms_subsample, ncellsperm_subsample,
                                             dims, twin, windt, dtovr)